In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import scipy as stats
import numpy as np

In [ ]:
restaruantpath= Path("csv_folder/restaurant_data.csv")
restaruantfile=pd.read_csv(restaruantpath,encoding="UTF-8")
restaruantfile


In [ ]:
# #Normalize Inspection Date columns
# query_df['inspection_date']=pd.to_datetime(query_df['inspection_date']).dt.normalize()
# print(query_df['inspection_date'].head())
# print(query_df.dtypes)


In [ ]:
#Delete unnessecary columns
restaurants_to_keep=['CAMIS','VIOLATION DESCRIPTION','DBA','BORO', 'ZIPCODE','CUISINE DESCRIPTION','INSPECTION DATE','GRADE']
restaruantfile=restaruantfile[restaurants_to_keep].copy()
#restaruantfile = restaruantfile.drop(columns=['BUILDING','STREET', 'PHONE', 'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA', 'RECORD DATE', 'GRADE DATE'])
restaruantfile.head()

In [ ]:
#Drop restaurants that don't have a grade (i.e., NaN) from the dataframe
#query_df['grade'].unique()
clean_grade_df = restaruantfile.dropna(subset=['GRADE'])



In [ ]:
#getting unique value lists which should be helpful later when making calls. this syntax can work for any column.
clean_grade_df['CUISINE DESCRIPTION'].unique().tolist()
clean_grade_df['BORO'].unique().tolist()

In [ ]:
#the code worked but i think we will have to run the dates through that filter code to get it sorted. I am holdingoff until class to do this.
clean_grade_df.sort_values('INSPECTION DATE',ascending=False)
clean_grade_df.head()

In [ ]:
#Restrict dataset to just September 2010 to September 2017
clean_grade_df['INSPECTION DATE'] = pd.to_datetime(clean_grade_df['INSPECTION DATE']).dt.date
clean_grade_df['Date']=clean_grade_df['INSPECTION DATE']
clean_grade_df['ZIPCODE'].fillna(-1,inplace=True)
clean_grade_df['ZIPCODE'] = clean_grade_df['ZIPCODE'].astype(int)
clean_grade_df

In [ ]:
#Need denominator to be unique restaurants

clean_grade_unique=clean_grade_df.drop_duplicates(subset=['CAMIS'])
clean_grade_unique.reset_index(drop=True, inplace=True)
clean_grade_unique

In [ ]:
# QUESTION 1: How many unique A, B, C restaurants were there in New York City from September 2010-September 2017?
gradecountscitywide=clean_grade_unique['GRADE'].value_counts()
gradecountscitywide

#Pie chart


In [ ]:
# Extract unique cuisine descriptions
cuisinedescriptions = clean_grade_unique['CUISINE DESCRIPTION'].unique()
cuisinedescriptions

In [ ]:
# Pare down list of cuisines for Chi-Square test

# Define a mapping dictionary for collapsing cuisines
cuisine_mapping = {
    'American': 'American',
    'Other': 'Other',
    'Korean': 'Asian',
    'Mexican': 'Latin American',
    'Latin American': 'Latin American',
    'Frozen Desserts': 'Desserts/Baked Goods',
    'Hotdogs': 'Fast Food',
    'Pizza': 'Pizza',
    'French':'European',
    'Bottled Beverages':'Coffee/Tea/Beverages',
    'Italian':'Italian',
    'Tex-Mex':'Latin American',
    'Japanese':'Asian',
    'Spanish':'European',
    'Coffee/Tea':'Coffee/Tea/Beverages',
    'Mediterranean':'European',
    'Caribbean':'Caribbean',
    'Middle Eastern': 'Middle Eastern',
    'German':'European',
    'Hamburgers':'Fast Food',
    'Sandwiches/Salads/Mixed Buffet':'Deli',
    'Vegan':'Special Diet',
    'Bagels/Pretzels':'Deli',
    'Jewish/Kosher':'Special Diet',
    'Filipino':'Asian',
    'Indian':'Indian',
    'Sandwiches':'Deli',
    'Seafood':'Seafood',
    'Chinese':'Asian',
    'Eastern European':'European',
    'Donuts':'Desserts/Baked Goods',
    'Soups/Salads/Sandwiches':'Deli',
    'Juice, Smoothies, Fruit Salads':'Juices/Smoothies',
    'African':'African',
    'Russian':'European',
    'Ethiopian':'African',
    'Mexican':'Latin American',
    'Steakhouse':'Steakhouse',
    'Irish':'European'
}

# Replace values in the 'CUISINES' column using the mapping dictionary
clean_grade_unique['CollapsedCUISINES'] = clean_grade_unique['CUISINE DESCRIPTION'].replace(cuisine_mapping)

cuisinedescriptionsnew = clean_grade_unique['CollapsedCUISINES'].unique()
cuisinedescriptionsnew



In [ ]:
#1b Is there an association between restaurant type and rating among unique restaurants? -Chi-Square
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(pd.crosstab(clean_grade_unique['CollapsedCUISINES'], clean_grade_unique['GRADE']))

print("Chi-square test statistic:", chi2)
print(f"p-value: {p:.5f}")
print("Degress of Freedom:", dof)

cuisinebyratingdf=pd.crosstab(clean_grade_unique['CollapsedCUISINES'], clean_grade_unique['GRADE'])
cuisinebyratingdf



In [ ]:
#Describe these 7 restaurants with C ratings: Where are they located? What are their names?

In [ ]:
# QUESTION 3: Number of grades by Borough
#restaurantcount = clean_grade_df.groupby(['BORO', 'DBA']).size().reset_index(name='count')


In [ ]:
# QUESTION 3: Number of grades by Borough
gradecounts= clean_grade_df.groupby(['BORO', 'GRADE']).size().reset_index(name='count')

boroughs = gradecounts['BORO']
counts = gradecounts['count']
grades = gradecounts['GRADE']

transposecounts = gradecounts.pivot(index='BORO', columns='GRADE', values='count').fillna(0)

transposecounts.plot(kind='bar', stacked=True)
plt.xlabel('Borough')
plt.ylabel('Number of Restaurant Grades')
plt.title('Number of Restaurant Grades by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

#Pie Chart for each Borough

In [ ]:
uniquerestboro=clean_grade_unique.groupby('BORO').count()
uniquerestboro

In [ ]:
violationcount= clean_grade_df.groupby(['BORO']).size().reset_index(name='count')
violationcount


In [ ]:
#QUESTION 3a: Do certain boroughs have more restaurant violations?
violationcount= clean_grade_df.groupby(['BORO']).size().reset_index(name='count')

camis_df1 = pd.DataFrame({
    'borough': ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
    'Count': [104, 35, 159, 62, 12]
})

violation_df2 = pd.DataFrame({
    'borough': ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
    'Count': [252, 91, 371, 119, 29]
})

# Display the original DataFrames
print("DataFrame 1:")
print(camis_df1)
print("\nDataFrame 2:")
print(violation_df2)

# Merge the DataFrames on 'borough'
merged_df = pd.merge(camis_df1, violation_df2, on='borough', suffixes=('_df1', '_df2'))

# Perform the division
merged_df['Count_Division'] = merged_df['Count_df2'] / merged_df['Count_df1']*100

# Display the result
print("\nMerged DataFrame with Division:")
print(merged_df)

#hello


#ratboroughs = ratcounting['Borough']
#ratcounts = ratcounting.groupby('Borough').count()['Complaint Type']


#rattransposecounts = ratcounting.pivot(index='Borough', columns=ratboroughs, values=ratcounts).fillna(0)

merged_df.plot(kind='bar', x='borough', y='Count_Division')
plt.xlabel('Borough')
plt.ylabel('Borough Restaurant Violations by Percentage')
plt.title(' Violation Percentage by Borough')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
#Do certain Zip Codes have higher restaurant ratings?
#zipcountscitywide=clean_grade_unique['ZIPCODE'].value_counts()
#zipcountscitywide

In [ ]:
print(clean_grade_df.columns)

BEGIN RAT DATA DO NOT PUT PIZZA DATA BELOW

In [ ]:
ratpath= Path("csv_folder/rat_sightings.csv")
ratfile=pd.read_csv(ratpath,encoding="UTF-8")
ratfile.head()

In [ ]:
#came up with way to easily filter the main DF by columns. This can easily be amended by adding column titles to columns_to_keep and rerunning the block.
#i took facility type out Facililty Type because it is only Nans. scroll down to see output confirming.
columns_to_keep=['Borough','Incident Zip','Created Date','Location Type','City','Status','Complaint Type']
clean_rats_df=ratfile[columns_to_keep].copy()
clean_rats_df

In [ ]:
#fixed the zip code formatting, had to fill NaNs but we can discuss dropping them maybe? Im having trouble picturing how it will effect the final project.
#next we need to run that code on the date here, i cant seem to locate it
clean_rats_df['Incident Zip'].fillna(-1,inplace=True)
clean_rats_df['Incident Zip'] = clean_rats_df['Incident Zip'].astype(int)
clean_rats_df['Created Date'] = pd.to_datetime(clean_rats_df['Created Date']).dt.date
clean_rats_df['Date']=clean_rats_df['Created Date']
clean_rats_df['ZIPCODE']=clean_rats_df['Incident Zip']
clean_rats_df

In [ ]:
clean_rats_df.columns

In [ ]:
#QUESTION 2: How many rat sightings were there in New York City from September 2010-September 2017?
clean_rats_df['Complaint Type'].value_counts()

In [ ]:
merged_df=pd.merge(clean_grade_df,clean_rats_df, on='Date')



In [ ]:
zip_merge=pd.merge(clean_grade_df,clean_rats_df, on='ZIPCODE')

In [ ]:
#QUESTION 4: Do certain boroughs have more rat sightings?
ratcounting=  clean_rats_df.groupby(['Borough']).size().reset_index(name='count')

#ratboroughs = ratcounting['Borough']
#ratcounts = ratcounting.groupby('Borough').count()['Complaint Type']


#rattransposecounts = ratcounting.pivot(index='Borough', columns=ratboroughs, values=ratcounts).fillna(0)

ratcounting.plot(kind='bar', x='Borough', y='count')
plt.xlabel('Borough')
plt.ylabel('Number of Rat Sightings')
plt.title('Number of Rat Sightings by Borough, 2010-2017')
plt.xticks(rotation=0) 
plt.tight_layout()
plt.show()

In [ ]:
correct_count=ratcounting['Borough'].pop(5)

In [ ]:
ratcounting

In [ ]:
#consider per capita data. Find population and divide the rat sightings by the number of people.

In [ ]:
borough_pops=pd.Series([{'BRONX':1471160},
                {'BROOKLYN':2736074},
                {'MANHATTAN':1694251},
                {'QUEENS':2405464},
                {'STATEN ISLAND':495747}],index=)
borough_pops

In [ ]:
dict_lis=[{'BRONX':1471160},
                {'BROOKLYN':2736074},
                {'MANHATTAN':1694251},
                {'QUEENS':2405464},
                {'STATEN ISLAND':495747}]

In [ ]:
[dict_lis]

In [ ]:
borough_count=clean_rats_df['Borough'].value_counts()
borough_count.pop('Unspecified')

In [ ]:
borough_count['BROOKLYN']

In [ ]:
?pd.Series